# Tutorial: Gli strumenti di base per il deep learning privato

Benvenuto! In questo primo tutorial cercheremo di spiegarti come funziona PySyft. PySyft nasce come soluzione alle problematiche di privacy e sicurezza dei dati nei sistemi di machine learning, fornendo strumenti al supporto della decentralizzazione dei modelli e la privatizzazione dei dati. Questa serie di notebook si configura come una guida passo dopo  passo per venire a conoscenza di nuovi sistemi e strumenti al servizio del deep learning su dati, modelli encriptati o privati in maniera decentralizzata. 



**Scopo:** In questi tutorial, non ci limiteremo esclusivamente a trattare in maniera teorica tecniche di decentralizzazione o crittografia dei dati, ma utilizzeremo PySyft per decentralizzare l'intero ecosistema attorno ai dati. Ciò comprende sia i database in cui i dati vengono archiviati, sia i modelli di reti neurali impiegate. Man mano che verranno implementate nuove funzionalità per PySyft, questi notebook verranno estesi con nuovi tutorial per spiegarne il loro funzionamento. 

Autore:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Traduzione ad opera di:
- Daniele Gadler - Twitter [@DanieleGadler](https://twitter.com/DanieleGadler)
- Andrea Provino 

## Struttura:

- Parte 1: Gli strumenti di base per il deep learning privato


## Why Take This Tutorial?

**1) Vantaggio competitivo nella ricerca di un lavoro**. Negli ultimi 20 anni, lo sviluppo tecnologico ha prodotto una mole di dati impressionante, riducendo progressivamente il tempo di raddoppio: quello necessario a produrre una quantità di dati pari a quella creata dall’inizio della storia umana al momento della misurazione.

Le nuove regolazioni sui dati, come il [GDPR](https://eugdpr.org/), hanno introdotto regolamenti sempre più stringenti sul passaggio e sulla manipolazione delle informazioni degli utent, motivo per cui oggigiorno le aziende devono fare particolare  attenzione all'utilizzo e all'analisi dei dati utente.

Queste regolamentazioni impongono delle grosse limitazioni ai data scientist, i queli si trovano a dover lavorare su piccoli dataset contenenti poche informazioni utili di alto valore. Apprendendo strumenti privacy-preserving per il deep learning, come PySyft, puoi restare anche tu al passo anche con i tempi e ottenerne un vantaggio per la tua carriera.

**2) Entrepreneurial Opportunities** -  Esistono moltissimi problemi sociali che potrebbero essere risolti o attenuati sfruttando tecnologie di deep learning. Attualmente, il problema è l’assenza o l'impossibilità di accedere ai dati in conformità con le regolazioni esistenti. Questi sistemi avrebbero bisogno di accedere a molte informazioni sensibili e private su persone con problemi di salute, la cui privacy andrebbe preservata ad ogni costo.

Imparare a utilizzare sistemi per il deep learning privato equivale quindi a fornire nuove opportunità di business per le startup, altrimenti non disponibili con sistemi di machine learning tradizionali. 


**3) Social Good** - Deep Learning can be used to solve a wide variety of problems in the real world, but Deep Learning on *personal information* is Deep Learning about people, *for people*. Learning how to do Deep Learning on data you don't own represents more than a career or entrepreneurial opportunity, it is the opportunity to help solve some of the most personal and important problems in people's lives - and to do it at scale.

## How do I get extra credit?

- Star PySyft on GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Make a Youtube video teaching this notebook!


... ok ... let's do this!

# Part -1: Prerequisites

- Know PyTorch - if not then take the http://fast.ai course and come back
- Read the PySyft Framework Paper https://arxiv.org/pdf/1811.04017.pdf! This will give you a thorough background on how PySyft is constructed which will help things make more sense.

# Part 0: Setup

To begin, you'll need to make sure you have the right things installed. To do so, head on over to PySyft's readme and follow the setup instructions. TLDR for most folks is.

- Install Python 3.6 or higher
- Install PyTorch 1.3
- Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r pip-dep/requirements.txt
- pip install -r pip-dep/requirements_udacity.txt
- python setup.py install udacity
- python setup.py test

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a GitHub Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [ ]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in PyTorch, then you're used to working with torch.Tensor objects like these!

In [ ]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some **other** machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using Torch tensors, we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")

Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [ ]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

And now - let's send our tensors to Bob!!

In [ ]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [ ]:
x_ptr

BOOM! Now Bob has two tensors! Don't believe me? Have a look for yourself!

In [ ]:
bob._objects

In [ ]:
z = x_ptr + x_ptr

In [ ]:
z

In [ ]:
bob._objects

Now notice something. When we called `x.send(bob)` it returned a new object that we called `x_ptr`. This is our first *pointer* to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata that pointers contain.

In [ ]:
x_ptr

Check out that metadata!

There are two main attributes specific to pointers:

- `x_ptr.location : bob`, the location, a reference to the location that the pointer is pointing to
- `x_ptr.id_at_location : <random integer>`, the id where the tensor is stored at location

They are printed in the format `<id_at_location>@<location>`

There are also other more generic attributes:
- `x_ptr.id : <random integer>`, the id of our pointer tensor, it was allocated randomly
- `x_ptr.owner : "me"`, the worker which owns the pointer tensor, here it's the local worker, named "me"


In [ ]:
x_ptr.location

In [ ]:
bob

In [ ]:
bob == x_ptr.location

In [ ]:
x_ptr.id_at_location

In [ ]:
x_ptr.owner

You may wonder why the local worker which owns the pointer is also a VirtualWorker, although we didn't create it.
Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we called `hook = sy.TorchHook()` and so you don't usually have to create it yourself.

In [ ]:
me = sy.local_worker
me

In [ ]:
me == x_ptr.owner

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [ ]:
x_ptr

In [ ]:
x_ptr.get()

In [ ]:
y_ptr

In [ ]:
y_ptr.get()

In [ ]:
z.get()

In [ ]:
bob._objects

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy! You can just use pointers like you would normal tensors!

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [ ]:
z = x + y

In [ ]:
z

And voilà! 

Behind the scenes, something very powerful happened. Instead of x and y computing an addition locally, a command was serialized and sent to Bob, who performed the computation, created a tensor z, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!

In [ ]:
z.get()

### Torch Functions

This API has been extended to all of Torch's operations!!!

In [ ]:
x

In [ ]:
y

In [ ]:
z = torch.add(x,y)
z

In [ ]:
z.get()

### Variables (including backpropagation!)

In [ ]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

In [ ]:
x = x.get()

In [ ]:
x

In [ ]:
x.grad

So as you can see, the API is really quite flexible and capable of performing nearly any operation you would normally perform in Torch on *remote data*. This lays the groundwork for our more advanced privacy preserving protocols such as Federated Learning, Secure Multi-Party Computation, and Differential Privacy !

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)